In [2]:
pip install -q openai azure-core azure-search-documents azure-storage-blob python-docx

StatementMeta(smallbronzefull, 1876, 3, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [4]:
%run Functions/ReferenceFunctions

StatementMeta(smallbronzefull, 1876, 34, Finished, Available, Finished)

In [7]:
from datetime import datetime
import json
from notebookutils import mssparkutils
import adal, datetime, json, re, time
from delta.tables import *
from pyspark.sql.types import *

from pyspark.sql.functions import *

import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

StatementMeta(smallbronzefull, 1876, 37, Finished, Available, Finished)

In [ ]:
TenantID = ""
SynapseSPNClientID = ""
KeyVaultName = ""
ClientIDKeyVaultSecretName = ""
AzureSQLJDBCURLSecretName = ""
# Step 1: Get the Azure SQL JDBC URL

tokenlibrary = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary  

AzureSQLjdbcurl = tokenlibrary.getSecret(f"{KeyVaultName}", f"{AzureSQLJDBCURLSecretName}", "ls_kv_dag")  


# Step 2: Get the Access Token

access_token,exec_statement,con = jdbc_authorization(KeyVaultName, AzureSQLjdbcurl, ClientIDKeyVaultSecretName, TenantID, SynapseSPNClientID, "SELECT 1") 

StatementMeta(smallbronzefull, 1876, 38, Finished, Available, Finished)

In [12]:
def log_activity(pid, stage, status,start,end, size,filename):
    duration = (end - start).total_seconds()
    throughput = size/duration
    start = start.strftime("%Y%m%d %H:%M:%S")
    end = end.strftime("%Y%m%d %H:%M:%S")
    stmt = con.createStatement()
    insert = f"INSERT INTO dbo.pipeline_activity(id,stage,file_name,stat,start_time,end_time,duration,size,throughput,) VALUES({pid},{stage},{filename},{status},{start},{end},{duration},{size},{throughput})"
    stmt.executeUpdate(insert)
    con.commit()
    print(f"logs updated for {filename} with id: {pid}")

StatementMeta(smallbronzefull, 1876, 42, Finished, Available, Finished)

In [13]:
import io
import uuid
from datetime import datetime, timedelta
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.storage.blob import BlobServiceClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField
)
from azure.storage.blob import (
    BlobServiceClient,
    BlobSasPermissions,
    generate_blob_sas,
)
from docx import Document 

StatementMeta(smallbronzefull, 1876, 43, Finished, Available, Finished)

In [ ]:
service_endpoint = ""
index_name       = "docxidx"
key              = ""
credential       = AzureKeyCredential(key)
vector_dims       = 1536
algo_name         = "hnsw-cosine"
profile_name      = "openai-ada-profile"

StatementMeta(smallbronzefull, 1876, 44, Finished, Available, Finished)

In [15]:
# Define the index fields
client = SearchIndexClient(service_endpoint, credential)
fields = [
    SimpleField(name="id",   type=SearchFieldDataType.String,key=True, sortable=True, filterable=True, facetable=True),
    SimpleField(name="file_name", type=SearchFieldDataType.String),
    
    SimpleField(name="page_number",type=SearchFieldDataType.Int32,sortable=True,filterable=True,facetable=False),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="content_vector",type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True,vector_search_dimensions=1536,vector_search_profile_name="my-vector-config"),
    SimpleField(name="storage_url",type=SearchFieldDataType.String,filterable=False,facetable=False,sortable=False)
]
 
vector_search = VectorSearch(
    profiles=[VectorSearchProfile(name="my-vector-config",
                                algorithm_configuration_name="my-algorithms-config")],
    algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
)

# Define semantic configuration
semantic_config = SemanticConfiguration(
    name="semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="file_name"),
        content_fields=[SemanticField(field_name="content")]
    )
)

# Add semantic search to the index
semantic_search = SemanticSearch(
    default_configuration_name="semantic-config",
    configurations=[semantic_config]
)

# Update the index definition
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

client.create_or_update_index(index)

StatementMeta(smallbronzefull, 1876, 45, Finished, Available, Finished)

In [ ]:
# ─── 1. CONFIG ────────────────────────────────────────────────────────────────
AZURE_OPENAI_ENDPOINT  = ""
AZURE_OPENAI_KEY       = ""
AZURE_OPENAI_API_VER   = "2024-10-21"
EMBED_DEPLOYMENT       = "emb"
 
SEARCH_ENDPOINT        = ""
SEARCH_API_KEY         = ""
SEARCH_INDEX_NAME      = "docxidx"
 
ADLS_CONNECTION_STRING = ""  # blob endpoint
ADLS_CONTAINER_NAME    = ""
ADLS_DIR_PREFIX        = ""
ADLS_STORAGE_ACCOUNT  = ""
ADLS_STORAGE_KEY = ""
 
TEXT_MIN_CHARS         = 5
 
# ─── 2. CLIENTS ───────────────────────────────────────────────────────────────
openai_client = AzureOpenAI(
    api_key        = AZURE_OPENAI_KEY,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_version    = AZURE_OPENAI_API_VER,
)
 
search_client = SearchClient(
    endpoint   = SEARCH_ENDPOINT,
    index_name = SEARCH_INDEX_NAME,
    credential = AzureKeyCredential(SEARCH_API_KEY)
)
 
blob_service  = BlobServiceClient.from_connection_string(ADLS_CONNECTION_STRING)
 
container     = blob_service.get_container_client(ADLS_CONTAINER_NAME)
 
# ─── 3. HELPERS ───────────────────────────────────────────────────────────────
def create_embedding(text: str, model: str = EMBED_DEPLOYMENT) -> list[float]:
    """Return the embedding vector for `text`."""
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding

def docx_paragraphs_from_bytes(doc_bytes: bytes):
    doc = Document(io.BytesIO(doc_bytes))
    seg_no = 0
    for para in doc.paragraphs:
        text = para.text.strip()
        if len(text) >= TEXT_MIN_CHARS:
            seg_no += 1
            yield seg_no, text

def make_sas_url(file_name: str) -> str:
    sas = generate_blob_sas(
        account_name  = ADLS_STORAGE_ACCOUNT,
        container_name= ADLS_CONTAINER_NAME,
        blob_name     = file_name,
        account_key   = ADLS_STORAGE_KEY,
        permission    = BlobSasPermissions(read=True),
        expiry        = datetime.utcnow() + timedelta(days=7),
    )
    return f"https://{ADLS_STORAGE_ACCOUNT}.blob.core.usgovcloudapi.net/{ADLS_CONTAINER_NAME}/{file_name}?{sas}"
 
# ─── 4. MAIN ──────────────────────────────────────────────────────────────────

def main():
    uploaded = skipped = 0
    start = datetime.utcnow()
    print("➜ Scanning ADLS container for DOCX files …")
    total_docx_parsed = 0
    total_docx = 0
    page_parsed = 0
    docx_size =0

    for blob in container.list_blobs(name_starts_with=ADLS_DIR_PREFIX):
     
        if not blob.name.lower().endswith(".docx"):
            continue
        print(f"\n➜ Processing {blob.name}")
        doc_bytes = container.download_blob(blob.name).readall()
        total_docx += 1
        docx_size += len(doc_bytes)
        for seg_no, text in docx_paragraphs_from_bytes(doc_bytes):
            doc_record = {
                "id"            : str(uuid.uuid4()),
                "file_name"     : blob.name,
                "page_number"   : seg_no,        
                "content"       : text,
                "content_vector": create_embedding(text),
                "storage_url"   : make_sas_url(blob.name),
            }
            result = search_client.upload_documents([doc_record])[0]
            status = "✓" if result.succeeded else "✗"
            if(status == "✓"):
                page_parsed += 1
            uploaded += int(result.succeeded)
            skipped  += int(not result.succeeded)
        if(page_parsed > 0):
            total_docx_parsed +=1
        print(f"Total page parsed {page_parsed}")
        page_parsed = 0
        end = datetime.utcnow()
        pid = str(uuid.uuid4())
        stage = "Bronze to Silver"
        status = "Convert DOCX to Ai"
        size = docx_size
        log_activity(pid, stage,blob.name, status,start,end, size)
 
    print(f"\nDone. {uploaded} segments indexed, {skipped} segments failed.")
    print(f"Total DOCX processed: {total_docx_parsed}\n Total DOCX in Raw: {total_docx}" )
    
 
if __name__ == "__main__":

    main()
 

StatementMeta(, 1876, -1, Cancelled, , Cancelled)